Initial query

In [ ]:
import requests
import json

url = "http://localhost:11434/api/chat"
payload = {
    "model": "deepseek-r1:1.5b",
    "messages": [{"role": "user", "content": "Tell me about QLoRA in 100 words or less."}],
    "stream": False
}
response = requests.post(url, data=json.dumps(payload))
print(response.json())

Now let's do it in python. 

First we need to install ollama with pip

In [ ]:
!pip install ollama

Next let's create a simple python query using the ollama library

In [ ]:
import ollama
response = ollama.chat(
    model="deepseek-r1:1.5b",
    messages=[
        {"role": "user", "content": "Why is QLoRA such a popular method for fine tuning. Explain this in 100 words or less."},
    ],
)
print(response["message"]["content"])

As you can see unsloth is an unown term to the default deepseek model and the results can be quite humorous. We will want to fix this by fine tuning the model and give it some information about the unsloth library. 
The first step in doing this is to convert the provided unsloth_documentation.pdf into a dataset for training.
For this task we can use Docling and LiteLLM.
To help visualize different outputs we'll also install colorama to color code out terminal outputs

In [ ]:
!pip install docling litellm colorama

Most Likely you will want to run this on GPU so you'll need to install pytorch as well. I used the latest version with CUDA 12.8 since I have a 5000 series GPU which is not supported by earlier CUDA versions. Please check version [compatability](https://developer.nvidia.com/cuda-gpus)

In [7]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128

Looking in indexes: https://download.pytorch.org/whl/cu128
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 GB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 609.6/609.6 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 726.9/726.9 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.4/260.4 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/292.1 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.2/39.2 MB 60.0 MB

You may also might need to update huggingface cache persmissions and pre-download the docling-models.

In [ ]:
sudo chown -R $(whoami) ~/.cache/huggingface
huggingface-cli download ds4sd/docling-models

Now We can being getting out data chunks from the pdf and save them to a list of chunks

In [8]:
from docling.document_converter import DocumentConverter
from docling.chunking import HybridChunker
from colorama import Fore, init

# Initialize colorama for cross-platform colored output
init()

converter = DocumentConverter()
doc = converter.convert("unsloth_documentation.pdf").document
chunker = HybridChunker()
chunks = chunker.chunk(dl_doc=doc)

contextualized_chunks = []
for i, chunk in enumerate(chunks): 
    print(Fore.YELLOW + f"Raw Text:\n{chunk.text[:300]}…" + Fore.RESET)
    contextualized_chunks.append(chunker.contextualize(chunk=chunk))
    print(Fore.LIGHTMAGENTA_EX + f"Contextualized Text:\n{contextualized_chunks[i][:300]}…" + Fore.RESET)

RuntimeError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
